In [25]:
import pandas as pd
import json

In [2]:
# Connect to database

from sqlalchemy import create_engine

URI="localhost"
PORT="5433"
DB = "jetaDb"
USER = "postgres"
PASSWORD = "00001234"
  
engine = create_engine("postgresql://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

/home/isaac/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [5]:
# Variables from query

start = "719"
destination = "603"
lineid = "31B"
start_time = "1532018880"

# http://localhost:8000/main/journeytime?source=767&destination=1914&lineid=39A&time=1530008000
# http://localhost:8000/main/journeytime?source=719&destination=603&lineid=31B&time=1532018880
# Rain should be gotten from api or else table

rain = 0.5

In [6]:
# Transform time into variables required from model

import time
from datetime import datetime, timedelta
from pytz import timezone

# Get Irish timezone (utc + daylight saving time (DST))
irish_time = timezone('Europe/Dublin')
print("Irish Standard Time: ", irish_time)

# Get unixtime as datetime object
dt_time = datetime.fromtimestamp(int(start_time), irish_time)
print("Datetime: ", dt_time)

# Get day of week -> Mon: 0, Sun: 6
weekday = dt_time.weekday()
print("Weekday: ", weekday)

# Create list with desired weekday filled.
week_dummies = [0] * 7
week_dummies[weekday] = 1
del week_dummies[2] # Delete wednesday - not included in model due to dummy var trap
print("Week dummies: ", week_dummies)

# Get arrivaltime in seconds
date = dt_time.date()
date_unixtime = time.mktime(date.timetuple())
seconds_since_midnight = int(time.mktime((dt_time - timedelta(seconds = date_unixtime)).timetuple()))
print("Seconds since midnight (arrival time): ", seconds_since_midnight)


Irish Standard Time:  Europe/Dublin
Datetime:  2018-07-19 17:48:00+01:00
Weekday:  3
Week dummies:  [0, 0, 1, 0, 0, 0]
Seconds since midnight (arrival time):  64080


In [7]:
# Model inputs

model_inputs = [seconds_since_midnight, rain] + week_dummies
model_inputs

[64080, 0.5, 0, 0, 1, 0, 0, 0]

In [8]:
# Get stop lists associated with this lineid, start stop and end stop

sql = """

SELECT * 
FROM main_routes 
WHERE routeid IN (
    SELECT UNNEST(routes) 
    FROM main_lines 
    WHERE main_lines.lineid = '{0}'
) 
AND '{1}' = ANY(main_routes.stopids) 
AND '{2}' = ANY(main_routes.stopids) 
;

""".format(lineid, start, destination)

routes = pd.read_sql(sql, engine)

# Lines.objects.all
# Routes.objects.all(routeid )

2018-07-19 17:51:33,289 INFO sqlalchemy.engine.base.Engine select version()
2018-07-19 17:51:33,291 INFO sqlalchemy.engine.base.Engine {}
2018-07-19 17:51:33,302 INFO sqlalchemy.engine.base.Engine select current_schema()
2018-07-19 17:51:33,303 INFO sqlalchemy.engine.base.Engine {}
2018-07-19 17:51:33,310 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-19 17:51:33,311 INFO sqlalchemy.engine.base.Engine {}
2018-07-19 17:51:33,314 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-19 17:51:33,316 INFO sqlalchemy.engine.base.Engine {}
2018-07-19 17:51:33,321 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2018-07-19 17:51:33,323 INFO sqlalchemy.engine.base.Engine {}
2018-07-19 17:51:33,330 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

In [9]:
routes

,routeid,direction,stopids,lineid
0,31B_45,2,"[707, 708, 731, 710, 711, 712, 713, 714, 715, ...",31B
1,31B_46,2,"[557, 558, 559, 560, 562, 563, 564, 565, 566, ...",31B


In [10]:
if routes.shape[0] > 1:
    print("Error: multiple possible routes.")
    print(routes)

Error: multiple possible routes.
  routeid  direction                                            stopids lineid
0  31B_45          2  [707, 708, 731, 710, 711, 712, 713, 714, 715, ...    31B
1  31B_46          2  [557, 558, 559, 560, 562, 563, 564, 565, 566, ...    31B


In [11]:
# Convert list of stopids to list

stop_list = routes['stopids'].tolist()[0]
print(stop_list)

[707, 708, 731, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 693, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 675, 620, 1172, 7569, 7608]


In [12]:
# Slice list by start and destination stop

journey_stops = stop_list[stop_list.index(int(start)):(stop_list.index(int(destination))+1)]
print(journey_stops)

[719, 720, 721, 693, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603]


In [13]:
# Remove duplicate values from list, while maintaining stop order.

from more_itertools import unique_everseen

journey_stops = list(unique_everseen(journey_stops))
print(journey_stops)

[719, 720, 721, 693, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603]


In [14]:
# Change each stopid into string
stringified = list(map(str, journey_stops))

# Zip ['0', .. 'n'] and ['1', .., 'n'] into list [('0', '1'), .., ('n - 1', 'n')]
# Join tuples to make ['1_2', .., 'n-1_n']
journey_segments = [ '_'.join(x) for x in zip(stringified[0:], stringified[1:])]
print(journey_segments)

['719_720', '720_721', '721_693', '693_585', '585_586', '586_587', '587_588', '588_589', '589_590', '590_591', '591_592', '592_593', '593_594', '594_595', '595_596', '596_597', '597_598', '598_599', '599_600', '600_601', '601_602', '602_603']


In [15]:
journey_segments

['719_720',
 '720_721',
 '721_693',
 '693_585',
 '585_586',
 '586_587',
 '587_588',
 '588_589',
 '589_590',
 '590_591',
 '591_592',
 '592_593',
 '593_594',
 '594_595',
 '595_596',
 '596_597',
 '597_598',
 '598_599',
 '599_600',
 '600_601',
 '601_602',
 '602_603']

In [16]:
# Select coefficient rows with these segment ids

sql2 = """

SELECT *
FROM main_coefficients
WHERE segment = ANY(ARRAY{0})

""".format(journey_segments)

coefficients = pd.read_sql(sql2, engine)

2018-07-19 17:51:43,831 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2018-07-19 17:51:43,833 INFO sqlalchemy.engine.base.Engine {'name': "\n\nSELECT *\nFROM main_coefficients\nWHERE segment = ANY(ARRAY['719_720', '720_721', '721_693', '693_585', '585_586', '586_587', '587_588', '588_589 ... (12 characters truncated) ... , '590_591', '591_592', '592_593', '593_594', '594_595', '595_596', '596_597', '597_598', '598_599', '599_600', '600_601', '601_602', '602_603'])\n\n"}
2018-07-19 17:51:43,864 INFO sqlalchemy.engine.base.Engine 

SELECT *
FROM main_coefficients
WHERE segment = ANY(ARRAY['719_720', '720_721', '721_693', '693_585', '585_586', '586_587', '587_588', '588_589', '589_590', '590_591', '591_592', '592_593', '593_594', '594_595', '595_596', '596_597', '597_598', '598_599', '599_600', '600_601', '601_602', '602_603'])


2018-07-19 17:51:43,865 INFO sql

In [17]:
coefficients

,segment,intercept,arrivaltime,rain,fri,mon,sat,sun,thu,tue
0,590_591,77.486822,-0.000242,-0.807937,2.784023,-2.164096,0.998181,0.305914,0.537931,0.233844
1,719_720,28.659691,-0.000026,2.750861,-1.398714,-0.801720,6.281082,-0.651956,1.611254,0.973545
2,720_721,38.750237,-0.000162,0.150589,0.191052,0.660358,3.838071,0.066923,3.515841,0.868272
3,596_597,45.322046,-0.000216,0.713827,1.321934,-0.531579,0.724431,-1.241643,-0.155107,0.441909
4,721_693,52.741514,-0.000209,-5.792332,0.420312,3.187275,4.070429,-2.148878,0.315138,6.174065
5,601_602,29.323675,-0.000059,0.105935,-0.040756,-0.834677,-0.501504,-0.146129,-0.196711,-0.397803
6,602_603,34.506467,-0.000097,1.088108,0.240132,-1.218228,-0.089297,-2.447772,-0.467658,-0.237343
7,599_600,80.053142,-0.000346,1.084783,3.091560,-2.905276,-5.970481,-7.696129,-0.718530,-0.130769
8,600_601,64.207143,-0.000222,-0.812051,1.912011,-2.989893,-2.218802,-3.472349,-2.421626,-0.497059
9,592_593,40.274701,-0.000038,0.309898,0.726911,-0.236873,0.390297,-2.661621,0.258928,-0.500827


In [18]:
# Sort values by journey_segment segmentid

coefficients['segment'] = coefficients['segment'].astype("category")
coefficients['segment'].cat.set_categories(journey_segments, inplace=True)
coefficients = coefficients.sort_values(["segment"])
coefficients

,segment,intercept,arrivaltime,rain,fri,mon,sat,sun,thu,tue
1,719_720,28.659691,-0.000026,2.750861,-1.398714,-0.801720,6.281082,-0.651956,1.611254,0.973545
2,720_721,38.750237,-0.000162,0.150589,0.191052,0.660358,3.838071,0.066923,3.515841,0.868272
4,721_693,52.741514,-0.000209,-5.792332,0.420312,3.187275,4.070429,-2.148878,0.315138,6.174065
21,693_585,94.348035,-0.000393,-4.955985,-3.283793,9.637122,14.871955,-12.194740,1.070398,5.882774
10,585_586,57.240436,-0.000029,-1.057336,1.713583,-3.537300,-2.999639,-1.885056,-0.369444,0.896235
15,586_587,77.492901,-0.000208,-2.150294,0.651187,-0.479671,-3.490271,-4.951155,-0.501558,0.042648
17,587_588,34.561868,-0.000058,-0.592260,0.615317,-1.513907,1.016488,1.931522,-0.971741,-1.145625
16,588_589,44.999030,-0.000060,-0.438758,1.021729,-0.219345,0.313111,-0.567743,0.576869,0.531379
13,589_590,24.561617,-0.000017,-0.669214,0.576181,0.426825,0.608861,-0.041710,-0.133508,0.460098
0,590_591,77.486822,-0.000242,-0.807937,2.784023,-2.164096,0.998181,0.305914,0.537931,0.233844


In [20]:
# Rearrange columns and set segment id as index

coefficients = coefficients[["segment", "intercept", "arrivaltime", "rain", "mon", "tue", "thu", "fri", "sat", "sun"]]
coefficients = coefficients.set_index('segment')
coefficients.head(5)

,intercept,arrivaltime,rain,mon,tue,thu,fri,sat,sun
segment,,,,,,,,,
719_720,28.659691,-0.000026,2.750861,-0.801720,0.973545,1.611254,-1.398714,6.281082,-0.651956
720_721,38.750237,-0.000162,0.150589,0.660358,0.868272,3.515841,0.191052,3.838071,0.066923
721_693,52.741514,-0.000209,-5.792332,3.187275,6.174065,0.315138,0.420312,4.070429,-2.148878
693_585,94.348035,-0.000393,-4.955985,9.637122,5.882774,1.070398,-3.283793,14.871955,-12.194740
585_586,57.240436,-0.000029,-1.057336,-3.537300,0.896235,-0.369444,1.713583,-2.999639,-1.885056


In [21]:
arrivaltime = model_inputs[0]
totaltraveltime = 0
segment_times = []

for i, rows in coefficients.iterrows():
    traveltime = (rows['intercept']
                  +(rows['arrivaltime']*arrivaltime)
                  +(rows['rain']*model_inputs[1])
                  +(rows['fri']*model_inputs[2])
                  +(rows['mon']*model_inputs[3])
                  +(rows['sat']*model_inputs[4])
                  +(rows['sun']*model_inputs[5])
                  +(rows['thu']*model_inputs[6])
                  +(rows['fri']*model_inputs[7]))
    
    segment_times.append((i, round(traveltime)))
    totaltraveltime += traveltime
    arrivaltime = model_inputs[0] + totaltraveltime # initial start time + sum of previous segment times
        
print('Arrival Time:', int(arrivaltime))
print('Total Travel Time:', int(totaltraveltime))
#print('Segment Times:', segment_times)
segment_times

Arrival Time: 64939
Total Travel Time: 859


[('719_720', 35.0),
 ('720_721', 32.0),
 ('721_693', 40.0),
 ('693_585', 82.0),
 ('585_586', 52.0),
 ('586_587', 60.0),
 ('587_588', 32.0),
 ('588_589', 41.0),
 ('589_590', 24.0),
 ('590_591', 62.0),
 ('591_592', 35.0),
 ('592_593', 38.0),
 ('593_594', 33.0),
 ('594_595', 25.0),
 ('595_596', 24.0),
 ('596_597', 32.0),
 ('597_598', 31.0),
 ('598_599', 29.0),
 ('599_600', 52.0),
 ('600_601', 47.0),
 ('601_602', 25.0),
 ('602_603', 29.0)]

In [22]:
total = 0
for i in segment_times:
    total += i[1]
    
print(total)

860.0


In [23]:
# Construct json

json_dict = {}
json_dict['arrivaltime'] = round(arrivaltime)
json_dict['totaltraveltime'] = round(totaltraveltime)
json_dict['segment_times'] = {i[0]:i[1] for i in segment_times}

# for i in segment_times:
#     json_dict['segment_times'][i[0]] = i[1]

In [26]:
json.dumps(json_dict)

'{"arrivaltime": 64939.0, "totaltraveltime": 859.0, "segment_times": {"719_720": 35.0, "720_721": 32.0, "721_693": 40.0, "693_585": 82.0, "585_586": 52.0, "586_587": 60.0, "587_588": 32.0, "588_589": 41.0, "589_590": 24.0, "590_591": 62.0, "591_592": 35.0, "592_593": 38.0, "593_594": 33.0, "594_595": 25.0, "595_596": 24.0, "596_597": 32.0, "597_598": 31.0, "598_599": 29.0, "599_600": 52.0, "600_601": 47.0, "601_602": 25.0, "602_603": 29.0}}'